Example taken from Google ML Crash Course:
https://developers.google.com/machine-learning/crash-course/feature-crosses/encoding-nonlinearity

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 

import tensorflow as tf 

pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

# pd.show_versions()

## Linear Problem

### Create the DataSet

50 data points per category

In [ ]:
N = 50

Create the random data set for sick trees

Scale and shift the (0, 1) range from np.random.random to (-5, 5) respectively (0, 5)

See also here: https://stackoverflow.com/questions/59389241/how-to-generate-a-random-float-in-range-1-1-using-numpy-random-rand

In [ ]:
max_x, min_x = 5, -5
range_size = (max_x - min_x)
sick_x = np.random.random([N]) * range_size + min_x

max_y, min_y = 0, -5
range_size = (max_y - min_y)
sick_y = np.random.random([N]) * range_size + min_y

Create the randomn data set for healthy trees

In [ ]:
max_x, min_x = 5, -5
range_size = (max_x - min_x)
healthy_x = np.random.random([N]) * range_size + min_x

max_y, min_y = 5, 0
range_size = (max_y - min_y)
healthy_y = np.random.random([N]) * range_size + min_y

Plot the data

In [ ]:
plt.figure(figsize=(10, 8))

plt.scatter(sick_x, sick_y)
plt.scatter(healthy_x, healthy_y)

plt.xlabel('x') 
plt.ylabel('y') 

plt.title("Training Data") 
plt.show() 

Create the Pandas DataFrame

In [ ]:
# combine the x and y arrays
sick_trees = np.vstack((sick_x, sick_y)).T
healthy_trees = np.vstack((healthy_x, healthy_x)).T

data_column_names = ['x', 'y']

df_sick = pd.DataFrame(data=sick_trees, columns=data_column_names)
df_sick['status'] = 1

df_healthy = pd.DataFrame(data=healthy_trees, columns=data_column_names)
df_healthy['status'] = 0

# append is deprecated since pandas 1.4.0
# train_df = df_sick.append(df_healthy, ignore_index=True)
train_df = pd.concat([df_sick, df_healthy], ignore_index=True)

# Shuffle the examples.
shuffled_train_df = train_df.reindex(np.random.permutation(train_df.index)) 
shuffled_train_df.head(100)

# Print the entire DataFrame
# print(df)

Build and Train Model Functions

In [ ]:
#@title Define the functions that build and train a model
def build_model(my_learning_rate):
  """Create and compile a simple linear regression model."""
  # Most simple tf.keras models are sequential.
  model = tf.keras.models.Sequential()

  # Add one linear layer to the model to yield a simple linear regressor.
  model.add(tf.keras.layers.Dense(30, input_shape=(30,2)))
  # model.add(tf.keras.layers.Dense(32, 
  #                                   input_shape = (x_train.shape[1],), 
  #                                   activation = 'sigmoid'))

  # Compile the model topography into code that TensorFlow can efficiently
  # execute. Configure training to minimize the model's mean squared error. 
  model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=my_learning_rate),
                loss="mean_squared_error",
                metrics=[tf.keras.metrics.RootMeanSquaredError()])

  return model               


def train_model(model, x, y, my_epochs, 
                my_batch_size=None, my_validation_split=0.1):
  """Feed a dataset into the model in order to train it."""

  history = model.fit(x=x,
                      y=y,
                      batch_size=my_batch_size,
                      epochs=my_epochs,
                      validation_split=my_validation_split)

  # Gather the model's trained weight and bias.
  trained_weight = model.get_weights()[0]
  trained_bias = model.get_weights()[1]

  # The list of epochs is stored separately from the 
  # rest of history.
  epochs = history.epoch
  
  # Isolate the root mean squared error for each epoch.
  hist = pd.DataFrame(history.history)
  rmse = hist["root_mean_squared_error"]

  return epochs, rmse, history.history   

print("Defined the build_model and train_model functions.")

Define plotting functions

In [ ]:
def plot_the_model(trained_weight, trained_bias, feature, label):
  """Plot the trained model against 200 random training examples."""

  # Label the axes.
  plt.xlabel(feature)
  plt.ylabel(label)

  # Create a scatter plot from 200 random points of the dataset.
  random_examples = training_df.sample(n=200)
  plt.scatter(random_examples[feature], random_examples[label])

  # Create a red line representing the model. The red line starts
  # at coordinates (x0, y0) and ends at coordinates (x1, y1).
  x0 = 0
  y0 = trained_bias
  x1 = random_examples[feature].max()
  y1 = trained_bias + (trained_weight * x1)
  plt.plot([x0, x1], [y0, y1], c='r')

  # Render the scatter plot and the red line.
  plt.show()


def plot_the_loss_curve(epochs, rmse):
  """Plot a curve of loss vs. epoch."""

  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Root Mean Squared Error")

  plt.plot(epochs, rmse, label="Loss")
  plt.legend()
  plt.ylim([rmse.min()*0.97, rmse.max()])
  plt.show()  

print("Defined the plot_the_model and plot_the_loss_curve functions.")

### Train

In [ ]:
# The following variables are the hyperparameters.
learning_rate = 0.01
epochs = 30
batch_size = 30

# Specify the feature and the label.
my_feature = "total_rooms"  # the total number of rooms on a specific city block.
my_label="median_house_value" # the median value of a house on a specific city block.
# That is, you're going to create a model that predicts house value based 
# solely on total_rooms.  

# Discard any pre-existing version of the model.
my_model = None

# Invoke the functions.
my_model = build_model(learning_rate)
weight, bias, epochs, rmse = train_model(my_model, 
                                        shuffled_train_df.drop('status', axis=1),
                                        shuffled_train_df[['status']],
                                        epochs, batch_size)

print("\nThe learned weight for your model is %.4f" % weight)
print("The learned bias for your model is %.4f\n" % bias )

plot_the_model(weight, bias, my_feature, my_label)
plot_the_loss_curve(epochs, rmse)

## Non-Linear Problem

Create the sample data set

Scale and shift the (0, 1) range from np.random.random to (-5, 5).

See also here: https://stackoverflow.com/questions/59389241/how-to-generate-a-random-float-in-range-1-1-using-numpy-random-rand

In [ ]:
N = 100
max_val, min_val = 5, -5
range_size = (max_val - min_val)  # 2
x = np.random.random([N]) * range_size + min_val
y = np.random.random([N]) * range_size + min_val

In [ ]:
plt.figure(figsize=(10, 8))

plt.scatter(x, y)

plt.xlabel('x') 
plt.ylabel('y') 

plt.title("Training Data") 
plt.show() 